In [43]:
import random
import pandas as pd
import numpy as np

In [11]:
def generate_tile():
    # probability of getting a 2 = 0.9, probability of getting a 4 = 0.1
    return (2 if random.random() < 0.9 else 4)


In [65]:
def merge_tiles(state):
    new_state = []
    # merging
    i = 0
    while i < len(state):
        if i < len(state) - 1 and state[i] == state[i+1]:
            new_state.append(state[i] * 2)
            i += 2
        else:
            new_state.append(state[i])
            i += 1
    return new_state

In [104]:
def generate_states():
    NEW_TILES = {2: 0.9, 4: 0.1}
    states = [[0], [2, 2], [2, 4], [4, 4]]
    def helper(cur_state):
        if cur_state[-1] == 2048:
            return
        else:
            # merge first
            merged_state = merge_tiles(cur_state)
            
            # generate a tile
            for value, prob in NEW_TILES.items():
                next_state = merged_state + [value]
                next_state.sort()
                
                if next_state not in states:
                    states.append(next_state)
                    helper(next_state)


    helper(states[1])
    helper(states[2])
    helper(states[3])

    return states

In [105]:
states = generate_states()

print(len(states))

3487


In [106]:
# doing things on states
states.sort(key=sum)

states_data = {
    'state': states,
    'sum': [sum(state) for state in states]
}

states_df = pd.DataFrame(states_data)
display(states_df)
states_df.to_csv("states.csv")

,state,sum
0,[0],0
1,"[2, 2]",4
2,"[2, 4]",6
3,"[4, 4]",8
4,"[2, 2, 4]",8
...,...,...
3482,"[2, 2, 16, 16, 2048]",2084
3483,"[4, 8, 8, 16, 1024, 1024]",2084
3484,"[2, 4, 8, 8, 16, 2048]",2086
3485,"[2, 4, 16, 16, 2048]",2086


In [115]:
# absorbing states
absorbing_states = [state for state in states if state[-1] == 2048]
print(len(absorbing_states))
# transient states
transient_states = [state for state in states if state[-1] != 2048]
print(len(transient_states))

26
[[0], [2, 2], [2, 4], [4, 4], [2, 2, 4], [2, 4, 4], [2, 8], [2, 2, 8], [4, 4, 4], [4, 8], [2, 4, 8], [2, 2, 4, 8], [4, 4, 8], [2, 4, 4, 8], [2, 8, 8], [2, 2, 8, 8], [4, 4, 4, 8], [2, 2, 16], [4, 8, 8], [2, 4, 16], [2, 4, 8, 8], [2, 2, 4, 16], [4, 4, 16], [4, 4, 8, 8], [2, 4, 4, 16], [2, 8, 16], [2, 2, 8, 16], [4, 4, 4, 16], [4, 8, 16], [2, 4, 8, 16], [2, 2, 4, 8, 16], [4, 4, 8, 16], [2, 4, 4, 8, 16], [2, 8, 8, 16], [2, 2, 8, 8, 16], [4, 4, 4, 8, 16], [2, 2, 16, 16], [4, 8, 8, 16], [2, 4, 16, 16], [2, 4, 8, 8, 16], [2, 4, 32], [2, 2, 4, 32], [4, 4, 16, 16], [2, 2, 4, 16, 16], [4, 4, 8, 8, 16], [4, 4, 32], [2, 4, 4, 32], [2, 8, 32], [2, 4, 4, 16, 16], [2, 8, 16, 16], [2, 2, 8, 32], [4, 4, 4, 32], [4, 8, 32], [4, 8, 16, 16], [2, 4, 8, 32], [2, 2, 4, 8, 32], [4, 4, 8, 32], [2, 4, 4, 8, 32], [2, 8, 8, 32], [2, 2, 8, 8, 32], [4, 4, 4, 8, 32], [2, 2, 16, 32], [4, 8, 8, 32], [2, 4, 16, 32], [2, 4, 8, 8, 32], [2, 2, 4, 16, 32], [4, 4, 16, 32], [4, 4, 8, 8, 32], [2, 4, 4, 16, 32], [2, 8, 16, 

In [118]:
def generate_markov_chain():
    NEW_TILES = {2: 0.9, 4: 0.1}
    transitions = np.zeros((len(states), len(states)))
    pre_initial_state = [0]
    next_states = {tuple(state): [] for state in states}

    # calculate transition probabilities for initial states
    for value0, prob0 in NEW_TILES.items():
        for value1, prob1 in NEW_TILES.items():
            initial_state = [value0, value1]
            initial_state.sort()

            if initial_state not in next_states[tuple(pre_initial_state)]:
                next_states[tuple(pre_initial_state)].append(initial_state)

            transitions[states.index(pre_initial_state)][states.index(initial_state)] += prob0 * prob1

    def helper():
        for cur_state in states:
            cur_state_idx = states.index(cur_state)
            if cur_state[-1] == 2048:
                transitions[cur_state_idx][cur_state_idx] = 1
            elif len(cur_state) == 1:
                continue
            else:
                # merge first
                merged_state = merge_tiles(cur_state)
                
                # generate a tile
                for value, prob in NEW_TILES.items():
                    next_state = merged_state + [value]
                    next_state.sort()
                    if next_state not in next_states[tuple(cur_state)]:
                        next_states[tuple(cur_state)].append(next_state)

                    cur_state_idx = states.index(cur_state)
                    next_state_idx = states.index(next_state)
                    transitions[cur_state_idx][next_state_idx] += prob
                    # print("Constructing from " + str(cur_state) + " to " + str(next_state)) 
                    # print("With probability = " + str(transitions[states.index(cur_state)][states.index(next_state)]))


    def make_transition_matrix(row_states, column_states):
        transition_matrix = np.zeros((len(row_states), len(column_states)))
        for cur_state in row_states:
            row_idx = row_states.index(cur_state)

            for next_state in next_states[tuple(cur_state)]:
                if next_state in column_states:
                    # print("Transition from " + str(cur_state) + " to " + str(next_state))
                    # print("With probability = " + str(transitions[states.index(cur_state)][states.index(next_state)]))
                    column_idx = column_states.index(next_state)

                    transition_matrix[row_idx][column_idx] = transitions[states.index(cur_state)][states.index(next_state)]
        return transition_matrix

    def make_fundamental_matrix():
        helper()
        t = len(transient_states)
        r = len(absorbing_states)
        Q = make_transition_matrix(transient_states, transient_states)
        R = make_transition_matrix(transient_states, absorbing_states)
        I = np.identity(r)
        P = np.block([[Q, R], [np.zeros((r, t)), I]])
        return t, r, Q, R, P


    return make_fundamental_matrix()
    


In [119]:
# matrix
t, r, Q, R, P = generate_markov_chain()

In [121]:
# calculating expected moves
a = np.identity(t) - Q
b = np.array([1]*len(transient_states))
avg_moves_df = pd.DataFrame({
    'Average Moves': np.linalg.solve(a, b) ,
})
display(avg_moves_df)

,Average Moves
0,939.809917
1,938.991736
2,938.082645
3,937.173554
4,937.173554
...,...
3456,1.000000
3457,2.000000
3458,1.000000
3459,1.000000


In [123]:
# calculating absorbing probabilities
a = np.identity(t) - Q
b = R
absorbing_probability_df = pd.DataFrame(np.linalg.solve(a, b) ,)
display(absorbing_probability_df)

,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.03522,0.347896,0.003913,0.003913,0.076875,0.290978,0.004247,0.032283,0.141673,0.039535,...,0.000047,0.000006,0.000002,1.913818e-07,5.383636e-08,5.383636e-08,6.627273e-09,5.981818e-09,1.472727e-09,8.181818e-11
1,0.03522,0.347896,0.003913,0.003913,0.076875,0.290978,0.004247,0.032283,0.141673,0.039535,...,0.000047,0.000006,0.000002,1.913818e-07,5.383636e-08,5.383636e-08,6.627273e-09,5.981818e-09,1.472727e-09,8.181818e-11
2,0.03522,0.347896,0.003913,0.003913,0.076875,0.290978,0.004247,0.032283,0.141673,0.039535,...,0.000047,0.000006,0.000002,1.913818e-07,5.383636e-08,5.383636e-08,6.627273e-09,5.981818e-09,1.472727e-09,8.181818e-11
3,0.03522,0.347896,0.003913,0.003913,0.076875,0.290978,0.004247,0.032283,0.141673,0.039535,...,0.000047,0.000006,0.000002,1.913818e-07,5.383636e-08,5.383636e-08,6.627273e-09,5.981818e-09,1.472727e-09,8.181818e-11
4,0.03522,0.347896,0.003913,0.003913,0.076875,0.290978,0.004247,0.032283,0.141673,0.039535,...,0.000047,0.000006,0.000002,1.913818e-07,5.383636e-08,5.383636e-08,6.627273e-09,5.981818e-09,1.472727e-09,8.181818e-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3456,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.900000,0.000000,0.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
3457,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,8.100000e-01,0.000000e+00,1.800000e-01,1.000000e-02
3458,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,9.000000e-01,0.000000e+00,1.000000e-01,0.000000e+00,0.000000e+00
3459,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,9.000000e-01,0.000000e+00,1.000000e-01,0.000000e+00


In [32]:
def simulate():
    state = [0]
    no_of_moves = 0
    while state[-1] != 2048:
        if len(state) == 1:
            # generate two random tiles
            state = []
            state.append(generate_tile())
            state.append(generate_tile())
        else:
            # merging
            state = merge_tiles(state) 

            # generate a new cell
            state.append(generate_tile())

        # disregard order
        state.sort()

        no_of_moves += 1
    absorbing_probabilities[tuple(state)] += 1
    return no_of_moves




no_of_trials = 1000000
absorbing_probabilities = {tuple(absorbing_state): 0 for absorbing_state in absorbing_states}
moves = {}

for _ in range(no_of_trials):
    move = simulate()
    
    if move in moves:
        moves[move] += 1
    else:
        moves[move] = 1

for item in absorbing_probabilities:
    absorbing_probabilities[tuple(item)] /= no_of_trials


In [33]:
# number of moves to win
moves = dict(sorted(moves.items()))
moves_data = {
    'No of Moves': moves.keys(),
    'Probability': [x / no_of_trials for x in moves.values()]
}

moves_df = pd.DataFrame(moves_data)
display(moves_df)
moves_df.to_csv("moves.csv")


,No of Moves,Probability
0,899,0.000001
1,901,0.000002
2,902,0.000007
3,903,0.000004
4,904,0.000009
...,...,...
73,973,0.000013
74,974,0.000006
75,975,0.000005
76,976,0.000005


In [35]:
# absorbing probabilities
# assert sum(absorbing_probabilities.values()) == 1.0, "wrong probability"
absorbing_states_data = {
    'absorbing state': absorbing_states,
    'absorbing probability': [absorbing_probabilities[tuple(state)] for state in absorbing_states],
    'sum': [sum(state) for state in absorbing_states]
}

absorbing_states_df = pd.DataFrame(absorbing_states_data)
display(absorbing_states_df)
absorbing_states_df.to_csv("absorbing_states.csv")

,absorbing state,absorbing probability,sum
0,"[2, 4, 4, 8, 2048]",0.035334,2066
1,"[2, 2, 8, 8, 2048]",0.347396,2068
2,"[4, 4, 4, 8, 2048]",0.003894,2068
3,"[2, 2, 16, 2048]",0.003870,2068
4,"[2, 4, 8, 8, 2048]",0.077021,2070
5,"[2, 4, 16, 2048]",0.291512,2070
6,"[4, 4, 8, 8, 2048]",0.004361,2072
7,"[4, 4, 16, 2048]",0.032268,2072
8,"[2, 2, 4, 16, 2048]",0.141260,2072
9,"[2, 4, 4, 16, 2048]",0.039471,2074
